# Import librairie et dataset

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import tqdm

from sklearn import datasets
from sklearn.datasets import load_iris


## titanic dataset xls

In [2]:
chemin = "cours-et-data/cours-et-data/"
nom_fichier = "titanic.xls"
df = pd.read_excel(chemin+nom_fichier)

#Suppression des colonnes non désiré
df = df.drop(['name','sibsp','parch','ticket','cabin','embarked','boat','body', 'home.dest'],axis='columns')

#Transformation des valeurs du sexe en 1 et 0
df["sex"].replace(['male','female'], value=[0,1], inplace=True)

#Suppression des valeurs null
df.dropna(axis='index', inplace=True)

## Dataset sklearn titanic

In [3]:
#Récupération du dataset de sklearn
df = sns.load_dataset('titanic')

#Suppression des colonnes non désiré
df = df.drop(['sibsp','parch','embarked','class','who','adult_male','deck','embark_town','alone'],axis='columns')

#Transformation des valeurs du sexe en 1 et 0
df["sex"].replace(['male','female'], value=[0,1], inplace=True)

#Transformation des valeurs de survie en 1 et 0
df["alive"].replace(['no','yes'], value=[0,1], inplace=True)

#Suppression des valeurs null
df.dropna(axis='index', inplace=True)

## iris dataset

In [4]:
#Récupération du dataset de sklearn
iris_data = load_iris()

#Création d'une dataframe à partir des données de la librairie
iris_features = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)

#Copie du dataframe
iris = iris_features

#Création d'une colonne avec les valeurs du nom de la fleure
iris['variety'] = pd.Categorical.from_codes(iris_data.target, iris_data.target_names)

# Récuparation x et y, séparation du jeu de donnée à 70%

In [5]:
#Association du y à la valeur survived
y = df["survived"]

#Asssociation du X aux caractéristiques (features) de la personne
X = df.drop('survived', axis="columns")

# Choix du modèle de machine learning

In [6]:
#Ici j'ai choisi perceptron mais on peut choisir d'autre modèle comme "from sklearn.linear_model import LinearRegression"
from sklearn.linear_model import Perceptron
model = Perceptron(eta0=0.1, random_state=0)

In [7]:
#Entraînement du modèle
classifieur = model.fit(X,y)

#Évaluation du modèle
model.score(X,y)

0.9901960784313726

# Plus loin

On peut choisir qu'elle partie on souhaite pour l'entrainement et le test.

In [8]:
#On import la fonction train_test_split() du module model_selection de sklearn
from sklearn.model_selection import train_test_split

#L'argument test_size=0.3 défini qu'on prend 30% du dataset pour le test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#On récupère les résultats de la prédiction qui se trouve dans X_test
y_pred = classifieur.predict(X_test)

#
good = (y_test == y_pred).sum()
bad = (y_test != y_pred).sum()
Resultat = good, "\n", bad 
print('Bons résultats %d' % good)
print('Erreurs: %d' % bad)
print(good/(bad+good))

Bons résultats 212
Erreurs: 3
0.986046511627907


#Ce code fonctionne si vous avez utilisé "LinearRegression"


def willyousurvive(model, pclass,sex, age):
    """Ce code prend en entrée le model LinearRegression,
    puis vous devez spécifier la classe que vous avez choisi,
    votre sexe et votre age
    Args :
    pclass: classe allant de 1 à 3
    sex : 0 pour homme et 1 pour femme
    age : integer
    """
    x = np.array([pclass, sex,age]).reshape(1,3)
    return model.predict_proba(x)

willyousurvive(model, 3, 0, 26)

# Comparaison different model

Ce code va comparer les différents modèles pour les comparer par la suite

In [14]:
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV, LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

liste_classifieur = [
    ["LogisticRegression", LogisticRegression(random_state=0)],
    ["RidgeClassifierCV", RidgeClassifierCV()],
    ["LinearRegression", LinearRegression()],
    ["SVC", SVC(probability=True, random_state=0)],  # Ajout de probability=True pour obtenir les probabilités
    ["LinearSVC", LinearSVC(random_state=0)],
    ["RandomForestClassifier", RandomForestClassifier(random_state=0)],
    ["GradientBoostingClassifier", GradientBoostingClassifier(random_state=0)],
    ["KNeighborsClassifier", KNeighborsClassifier()],
    ["GaussianNB", GaussianNB()],
    ["Perceptron", Perceptron(eta0=0.1, random_state=0)],
    ["DecisionTreeClassifier", DecisionTreeClassifier(random_state=0)]
]

#Entraînement du modèle
classifieur = model.fit(X,y)

#Évaluation du modèle
model.score(X,y)

#Création d'un dictionnaire pour enregistrer les résultats
dictionnaire_evaluation = {}

#Séparation du train et test à 70%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 

#Pour chaque classifieur de ma liste
for nom, classifieur in tqdm.tqdm(liste_classifieur):
    print("---------------------------------------------------------------- \n",nom,"\n")
    try:
        #j'entraine le jeu de donner sur le classifieur
        classifieur.fit(X_train, y_train)

        #Je sors ma prédiction
        y_pred = classifieur.predict(X_test)

        #Je vérifie les résultats
        good = (y_test == y_pred).sum()
        bad = (y_test != y_pred).sum()
        Resultat = good, "\n", bad 
        print('Bons résultats %d' % good)
        print('Erreurs: %d' % bad)
        print(good/(bad+good))

        # Calculer les métriques
        score = accuracy_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # Enregistrer les résultats dans le dictionnaire
        dictionnaire_evaluation[nom] = {
            "score": float(score),
            "VP": int(tp),
            "FP": int(fp),
            "FN": int(fn),
            "VN": int(tn)
        }
    except Exception as e:
        print(f"Erreur avec le classifieur {nom}: {e}")


  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\Loup\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\Loup\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 45%|████▌     | 5/11 [00:00<00:00, 45.83it/s]

---------------------------------------------------------------- 
 LogisticRegression 

Bons résultats 215
Erreurs: 0
1.0
---------------------------------------------------------------- 
 RidgeClassifierCV 

Bons résultats 215
Erreurs: 0
1.0
---------------------------------------------------------------- 
 LinearRegression 

Bons résultats 1
Erreurs: 214
0.004651162790697674
Erreur avec le classifieur LinearRegression: Classification metrics can't handle a mix of binary and continuous targets
---------------------------------------------------------------- 
 SVC 

Bons résultats 140
Erreurs: 75
0.6511627906976745
---------------------------------------------------------------- 
 LinearSVC 

Bons résultats 215
Erreurs: 0
1.0
---------------------------------------------------------------- 
 RandomForestClassifier 



100%|██████████| 11/11 [00:00<00:00, 33.82it/s]

Bons résultats 215
Erreurs: 0
1.0
---------------------------------------------------------------- 
 GradientBoostingClassifier 

Bons résultats 215
Erreurs: 0
1.0
---------------------------------------------------------------- 
 KNeighborsClassifier 

Bons résultats 146
Erreurs: 69
0.6790697674418604
---------------------------------------------------------------- 
 GaussianNB 

Bons résultats 215
Erreurs: 0
1.0
---------------------------------------------------------------- 
 Perceptron 

Bons résultats 125
Erreurs: 90
0.5813953488372093
---------------------------------------------------------------- 
 DecisionTreeClassifier 

Bons résultats 215
Erreurs: 0
1.0


In [15]:
print(dictionnaire_evaluation)

import json

#Chemin du fichier JSON au même dossier que le code
fichier_json = "cours-et-data/cours-et-data/resultats_evaluation.json"


#On écrit dans le fichier JSON
with open(fichier_json, 'w') as f:
    json.dump(dictionnaire_evaluation, f, indent=4)

{'LogisticRegression': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}, 'RidgeClassifierCV': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}, 'SVC': {'score': 0.6511627906976745, 'VP': 30, 'FP': 15, 'FN': 60, 'VN': 110}, 'LinearSVC': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}, 'RandomForestClassifier': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}, 'GradientBoostingClassifier': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}, 'KNeighborsClassifier': {'score': 0.6790697674418604, 'VP': 50, 'FP': 29, 'FN': 40, 'VN': 96}, 'GaussianNB': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}, 'Perceptron': {'score': 0.5813953488372093, 'VP': 0, 'FP': 0, 'FN': 90, 'VN': 125}, 'DecisionTreeClassifier': {'score': 1.0, 'VP': 90, 'FP': 0, 'FN': 0, 'VN': 125}}


In [12]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

##Comme nous savons que le 1 c'est les personnes qui vivent et le 0 les morts, on peut écrire ceci
nom_classes = ["vie", "meurt"]
report = classification_report(y_test, y_pred, target_names=nom_classes, digits=4)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       125
           1       1.00      1.00      1.00        90

    accuracy                           1.00       215
   macro avg       1.00      1.00      1.00       215
weighted avg       1.00      1.00      1.00       215

              precision    recall  f1-score   support

         vie     1.0000    1.0000    1.0000       125
       meurt     1.0000    1.0000    1.0000        90

    accuracy                         1.0000       215
   macro avg     1.0000    1.0000    1.0000       215
weighted avg     1.0000    1.0000    1.0000       215

